In [ ]:
import scipy
import random
import pickle
import plotly
import numpy as np
import pandas as pd
import cmdstanpy as stan
import plotly.express as px
from scipy.io import loadmat
from numpy import pi, sin, cos
from scipy.special import gamma
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import scipy.integrate as integrate
from scipy.spatial.distance import pdist, squareform
import matplotlib.patches as patches

import warnings
warnings.filterwarnings('default')

import logging
logger = logging.getLogger('cmdstanpy')
logger.setLevel(logging.ERROR)

%matplotlib inline

#if this is the first time of running this script, please uncomment the following line
stan.install_cmdstan(version='2.31.0')

# Import the data

In [ ]:
Raw_data_folder = 'Example_data'

df_raw_data = pd.read_csv(Raw_data_folder+'/Example_data.csv', index_col=[0,1,2,3,4,5])


In [ ]:
df_raw_data

# Calculate the pairwise distance matrix

In [ ]:
corr_dmat = squareform(pdist(df_raw_data))

# display the pairewise distance matrix
display(pd.DataFrame(corr_dmat))

In [ ]:

# normalize the distance matrix 
corr_dmat = 2.0*corr_dmat/np.max(corr_dmat)
print(corr_dmat.shape)
N = corr_dmat.shape[0]
print(N)

# visualize the pairwise distance matrix
fig, axs = plt.subplots(2, figsize=(3,5))
axs[0].imshow(corr_dmat)
axs[1].hist(corr_dmat[np.triu_indices(N, 1)])
fig.tight_layout()

# Initialize the STAN for hyperboloic MDS

In [ ]:
def d_lor(t1, t2, E1, E2):
    return np.arccosh(t1*t2 - np.dot(E1, E2))

def get_poin(fit):
    ts = np.sqrt(1.0 + np.sum(np.square(fit.euc), axis=1))
    return (fit.euc.T / (ts + 1)).T
    ts = np.sqrt(1.0 + np.sum(np.square(fit), axis=1))
    return (fit.T / (ts + 1)).T

#returns embedding distance matrix from optimization fit
def get_embed_dmat(fit):
    N = fit.euc.shape[0]
    fit_ts = np.sqrt(1.0 + np.sum(np.square(fit.euc), axis=1))

    fit_mat = np.zeros((N, N))

    for i in np.arange(N):
        for j in np.arange(i+1,N):
            fit_mat[i][j] = d_lor(fit_ts[i], fit_ts[j], fit.euc[i], fit.euc[j])
            fit_mat[j][i] = fit_mat[i][j]
            
    return fit_mat

#return negative log likelihood of fit
def MDS_lkl(fit, dmat):
    lkl = 0;
    N = fit.sig.shape[0]
    
    sigs = fit.sig
    lam = fit.stan_variable('lambda')
    emb_mat = get_embed_dmat(fit)
    
    for i in np.arange(N):
        for j in np.arange(i+1, N):
            seff = sigs[i]**2 + sigs[j]**2
            lkl += ((dmat[i][j] - emb_mat[i][j]/lam)**2 / (2.0*seff)) + 0.5*np.log(seff*2.0*np.pi)
    return lkl

#input: optimization fit and distance matrix
def BIC(fit, dmat):
    N,D = fit.euc.shape
    n = 0.5*N*(N-1)
    k = N*D + N + 1.0 - 0.5*D*(D-1)
    
    return k*np.log(n) + 2.0*MDS_lkl(fit,dmat)

def h_get_dmat(p_coords):
    N = p_coords.shape[0]
    dists = np.zeros((N, N))
    
    for i in np.arange(N):
        for j in np.arange(i+1, N):
            dists[i][j] = poincare_dist(p_coords[i], p_coords[j])
            dists[j][i] = dists[i][j]
    return dists

#returns hyperbolic distance between vectors in poincare ball
def poincare_dist(v1, v2):
    sq = np.sum(np.square(v1-v2))
    r1 = np.sum(np.square(v1))
    r2 = np.sum(np.square(v2))
    inv = 2.0*sq/((1.0-r1)*(1.0-r2))
    return np.arccosh(1.0 + inv)

In [ ]:
# Compile
ltz_model = stan.CmdStanModel(stan_file='lorentz.stan')

In [ ]:
# Choose a dimension to embedd the data

dim_choose = 3
dat={'N':N, 'D':dim_choose, 'deltaij':corr_dmat}
model = ltz_model.optimize(data=dat, iter=250000, algorithm='LBFGS', tol_rel_grad=1e3, show_console=True, refresh=1000)

In [ ]:
# Get the curvature from the embedding
print(model.stan_variable('lambda'))

# Get the coordinates from the embedding
poincare = get_poin(model)
df_data_embedded = pd.DataFrame(poincare, columns = ['X','Y','Z'], index = df_raw_data.index)
df_data_embedded

In [ ]:
# check the performance of the embedding by comparing the pairwise distance before and after the embedding
plt.figure(figsize=(6,6))
h_dmat = h_get_dmat(poincare)
x = np.arange(N**2).reshape(N, N)
dist = corr_dmat[np.triu_indices(N, 1)]
h_dist = h_dmat[np.triu_indices(N, 1)]

plt.scatter(dist,h_dist/model.stan_variable('lambda'),s=1)
plt.plot([0,2],[0,2])
plt.axis('square')
plt.title('Shepard Diagram Hyperbolic Embedding')
plt.xlabel('True Distance')
plt.ylabel('Embedded Distance')

# Visualize the embedding

In [ ]:
fig1 = px.scatter_3d(df_data_embedded, x='X', y='Y', z='Z',
                    color=df_data_embedded.index.get_level_values(2))

theta = np.linspace(0, 2*np.pi, 120)
phi = np.linspace(0, np.pi, 60)
u , v = np.meshgrid(theta, phi)
xs = np.cos(u)*np.sin(v)
ys = np.sin(u)*np.sin(v)
zs = np.cos(v)
x = []
y = []
z = []
for t in [theta[10*k] for k in range(12)]:  # meridians:
    x.extend(list(np.cos(t)*np.sin(phi))+[None])# None is inserted to mark the end of a meridian line
    y.extend(list(np.sin(t)*np.sin(phi))+[None]) 
    z.extend(list(np.cos(phi))+[None])
    
for s in [phi[6*k] for k in range(10)]:  # parallels
    x.extend(list(np.cos(theta)*np.sin(s))+[None]) # None is inserted to mark the end of a parallel line 
    y.extend(list(np.sin(theta)*np.sin(s))+[None]) 
    z.extend([np.cos(s)]*120+[None])
fig1.add_scatter3d(x=x, y=y, z=z, mode='lines', line_width=2, line_color='rgb(160,160,160)')

fig1.update_layout(
    height=800,
    template='simple_white',
    title_text='Hyperbolic embedding' 
)

#fig1.write_html('figure_3D.html', auto_open=True)